- follow the tasks 4.8
- exclude infinite values
- new column max_order
- new column customer_type
- new column spender_type

# 1) Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r"C:\Users\rieke\Data Analytics\Achievment 4\02-2023_Instacart Basket Analysis\02 Data"

In [3]:
path

'C:\\Users\\rieke\\Data Analytics\\Achievment 4\\02-2023_Instacart Basket Analysis\\02 Data'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged2.pkl'))

In [ ]:
ords_prods_merge.head()

In [6]:
ords_prods_merge = ords_prods_merge[ords_prods_merge['prices'] < 500]

In [ ]:
ords_prods_merge.head(15)

In [8]:
ords_prods_merge['prices'].describe()

count    3.239973e+07
mean              NaN
std      0.000000e+00
min      1.000000e+00
25%      4.199219e+00
50%      7.398438e+00
75%      1.129688e+01
max      2.500000e+01
Name: prices, dtype: float64

# 2) In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [9]:
ords_prods_merge.groupby('product_name')

In [10]:
ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# 3) Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

first of all we can see all departments are included now - not only a subset of them. They are displayed in full & ascending order. The most famous product group, having the highest average orders is the 21 (which was also not included in the subset) and of course the averages of all groups also contained in the subset differs in the entire df as there are much more information on each single group. In general we can say, that there are regular orders over the full range of departments with an average between 15 and 23.

# 4) Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

loyality flag was already included during the excercise

insert new col. max_order, group by user, transform max value (use numpy library) of order_nr into new col. max_order 


In [11]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

use loc() Function to specify flag


In [12]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [13]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [14]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [15]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15874128
Loyal customer      10282763
New customer         6242841
Name: loyalty_flag, dtype: int64

# 5) The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [16]:
ords_prods_merge.groupby('loyalty_flag')

In [17]:
# group by department, aggregate orders by mean, min, max
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.773438  1.0  25.0
New customer      7.800781  1.0  25.0
Regular customer  7.796875  1.0  25.0

mean of purchased products per customer group does not show relevant differences

# 6) The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
- If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
- If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [18]:
# set up new column with average prices
ords_prods_merge['avg_item_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.average)

In [ ]:
ords_prods_merge[['user_id', 'prices', 'avg_item_price']].head(30)

In [20]:
ords_prods_merge.loc[ords_prods_merge['avg_item_price']<10, 'spender_type'] = 'Low spender'

In [21]:
ords_prods_merge.loc[ords_prods_merge['avg_item_price']>10, 'spender_type'] = 'High spender'

In [22]:
ords_prods_merge['spender_type'].value_counts(dropna = False)

Low spender     32279618
High spender      119088
NaN                 1026
Name: spender_type, dtype: int64

NaN < 5%, therefore I am going to ignore this

# 7) In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows

In [23]:
# set up new column with order frequency
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
ords_prods_merge[['user_id', 'days_since_prior_order', 'order_frequency']].head(15)

- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [25]:
ords_prods_merge.loc[ords_prods_merge['order_frequency']>20, 'cust_type'] = 'Non-frequent customer'

In [26]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency']>10) & (ords_prods_merge['order_frequency'] <=20),'cust_type'] = 'Regular customer'

In [27]:
ords_prods_merge.loc[ords_prods_merge['order_frequency']<=10, 'cust_type'] = 'Frequent customer'

In [28]:
ords_prods_merge['cust_type'].value_counts(dropna = False)

Frequent customer        21556644
Regular customer          7207340
Non-frequent customer     3635743
NaN                             5
Name: cust_type, dtype: int64

In [29]:
df_nan = ords_prods_merge[ords_prods_merge['cust_type'].isnull() == True]

In [ ]:
df_nan

In [32]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32399732 entries, 0 to 32404858
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int32   
 1   user_id                 int32   
 2   order_number            int8    
 3   orders_day_of_week      int8    
 4   order_time_of_day       int8    
 5   days_since_prior_order  float16 
 6   product_id              int32   
 7   add_to_cart_order       int32   
 8   reordered               int8    
 9   product_name            object  
 10  aisle_id                int32   
 11  department_id           int8    
 12  prices                  float16 
 13  _merge                  category
 14  price_range             object  
 15  busiest_days            object  
 16  busiest_period_of_day   object  
 17  max_order               int8    
 18  loyalty_flag            object  
 19  avg_item_price          float16 
 20  spender_type            object  
 21  order_

In [31]:
ords_prods_merge.to_pickle(os.path.join(path, 'Prepared Date', ords_prods_merge_wo_inf.pkl))

NameError: name 'ords_prods_merge_inf' is not defined